In [1]:
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('D:\GITHUB\SpamEmailClassifier\data\CaptstoneProjectData_2025.csv')
# df = df.applymap(lambda x: x.replace('\r', '').replace('\n', '') if isinstance(x, str) else x)
df.fillna("",inplace=True)
df['row_id'] = range(len(df))
print(len(df))


<>:9: SyntaxWarning: invalid escape sequence '\G'
<>:9: SyntaxWarning: invalid escape sequence '\G'
C:\Users\yck\AppData\Local\Temp\ipykernel_19232\1139296043.py:9: SyntaxWarning: invalid escape sequence '\G'
  df = pd.read_csv('D:\GITHUB\SpamEmailClassifier\data\CaptstoneProjectData_2025.csv')
2025-05-08 14:00:29,851 - INFO - NumExpr defaulting to 12 threads.


2576


C:\Users\yck\AppData\Local\Temp\ipykernel_19232\1139296043.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("",inplace=True)


In [2]:
import re

df["http_count"] = df['Body'].str.count('http://')
df["https_count"] = df['Body'].str.count('https://')
df["is_body_empty"] = df['Body'].isna()
df["is_subject_empty"] = df['Subject'].isna()

df["paragraph_count"] = df['Body'].str.count('\n')
def extract_links(text):
    #
    if text is not None and isinstance(text,str):
        return re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
df["links"] = df['Body'].apply(extract_links)

df["subject_length"] = df['Subject'].str.len()
df["body_length"] =df['Body'].str.len()

def find_max_length_in_list(lst):
    if lst is not None and len(lst)>0:
        return max(len(item) for item in lst)
    else:
        return 0
df["max_link_length"] = df["links"].apply(find_max_length_in_list)
df["link_counts"] = df["links"].apply(lambda x:len(x) if x is not None else 0)

In [3]:
import re
import unicodedata


COMMON_PUNCTUATION = set([
    '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
    ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~'
])


def detect_suspicious_non_latin(text):
    suspicious_chars = []

    for char in text:
        if char.isascii():
            if char in COMMON_PUNCTUATION or char.isalnum() or char.isspace():
                continue
            else:
                suspicious_chars.append((char, 'Unusual ASCII'))
                continue

        try:
            name = unicodedata.name(char)
            if 'LATIN' not in name:
                suspicious_chars.append((char, name))
        except ValueError:
            suspicious_chars.append((char, 'No Unicode Name'))
    return len(suspicious_chars),len(set(suspicious_chars))
# Apply the function to the 'Body' column
df[["non_latin_count","non_latin_distinct_count"]] = df['Body'].apply(lambda x: detect_suspicious_non_latin(x) if isinstance(x,str) else 0).apply(pd.Series)
# df[['Body','detect_suspicious_non_latin']].sort_values(by='detect_suspicious_non_latin',ascending=False)
# ... existing code ...

In [4]:
import emoji

df["emoji_count"] = df['Body'].apply(lambda x: emoji.emoji_count(x) if isinstance(x,str) else 0)
df["emoji_distinct_count"] = df['Body'].apply(lambda x: len(emoji.distinct_emoji_list(x)) if isinstance(x,str) else 0)
df["emoji_list"] = df['Body'].apply(lambda x: emoji.emoji_list(x) if isinstance(x,str) else 0)

# print(emoji.emoji_count(email_content))
# print(len(emoji.distinct_emoji_list(email_content)))

In [5]:
df[["non_latin_distinct_count","Body"]].sort_values(by='non_latin_distinct_count',ascending=False)

,non_latin_distinct_count,Body
2446,135,"Hello, I am Manuel Franco, you have a donation..."
868,79,Invoice Due: October 12 2022\r\nAmount: USD 67...
426,33,\r\n\r\nYoulhavecalnewiVоiϲе-Nоtеl\r\n\r\nSend...
1713,30,\r\n\r\nUVIC Ѕυррогτ™®\r\nⅮυе τо геϲеɴτ υрɡгаԁ...
312,25,Notice: This message was sent from outside the...
...,...,...
990,0,Notice: This message was sent from outside the...
989,0,Notice: This message was sent from outside the...
988,0,Notice: This message was sent from outside the...
987,0,Notice: This message was sent from outside the...


In [6]:
selected_column = [ 'http_count', 'https_count',
       'is_body_empty', 'is_subject_empty', 'paragraph_count',
       'subject_length', 'body_length', 'max_link_length', 'link_counts',
       'non_latin_count', 'non_latin_distinct_count', 'emoji_count',
       'emoji_distinct_count']

df_for_kmeans = df[selected_column]


In [7]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, random_state=0)
kmeans.fit(df_for_kmeans)


KMeans(n_clusters=10, random_state=0)

In [8]:
df['label_kmeans'] = kmeans.labels_

In [9]:
df[df.emoji_distinct_count>2]

,Subject,Body,Unnamed: 2,Unnamed: 3,row_id,http_count,https_count,is_body_empty,is_subject_empty,paragraph_count,...,subject_length,body_length,max_link_length,link_counts,non_latin_count,non_latin_distinct_count,emoji_count,emoji_distinct_count,emoji_list,label_kmeans
351,Alternative Delivery Options. There might be s...,UPS features and services are a few clicks awa...,,,351,12,1,False,False,49,...,136,8014,694,13,186,4,3,3,"[{'match_start': 2493, 'match_end': 2494, 'emo...",6
393,Fulfillment delays in our Canadian warehouse: ...,Canada Post Parcel Division\r\n\r\n\r\n[http:/...,,,393,15,0,False,False,72,...,65,2393,118,14,4,4,3,3,"[{'match_start': 1099, 'match_end': 1100, 'emo...",0
563,,36 yrs divorced mom💋ready 4Fun🍑_xx69_👉 www.fcs...,,,563,0,0,False,False,2,...,0,56,0,0,3,3,3,3,"[{'match_start': 19, 'match_end': 20, 'emoji':...",9
604,Yellow Friday Sale starts TODAY! Savings start...,Notice: This message was sent from outside the...,,,604,0,25,False,False,48,...,59,3244,135,23,3,3,3,3,"[{'match_start': 1052, 'match_end': 1053, 'emo...",8
1713,Case_No: -045733452346,\r\n\r\nUVIC Ѕυррогτ™®\r\nⅮυе τо геϲеɴτ υрɡгаԁ...,,,1713,1,0,False,False,13,...,22,416,47,1,145,30,3,3,"[{'match_start': 16, 'match_end': 17, 'emoji':...",1
2223,Closing Of Email Address Notice!,cPanel\r\n\r\n\r\n\r\nSender\r\n\r\n Ac...,,,2223,0,2,False,False,57,...,32,1897,371,2,4,3,4,3,"[{'match_start': 230, 'match_end': 231, 'emoji...",0
